In [3]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

class CARTDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth

    def fit(self, X, y):
        label_encoder = LabelEncoder()
        y_encoded = label_encoder.fit_transform(y)
        self.class_names = label_encoder.classes_
        self.feature_names = np.array(['Outlook', 'Temp', 'Humidity', 'Wind'])
        self.tree = self._build_tree(X, y_encoded, depth=0)

    def _build_tree(self, X, y, depth):
        num_samples, num_features = X.shape
        num_classes = len(np.unique(y))

        if depth == self.max_depth or num_classes == 1:
            return {'class': np.argmax(np.bincount(y))}

        best_split = self._find_best_split(X, y, num_samples, num_features)

        left_indices = best_split['left_indices']
        right_indices = best_split['right_indices']
        left_tree = self._build_tree(X[left_indices], y[left_indices], depth + 1)
        right_tree = self._build_tree(X[right_indices], y[right_indices], depth + 1)

        return {'feature_index': best_split['feature_index'],
                'threshold': best_split['threshold'],
                'left': left_tree,
                'right': right_tree}

    def _find_best_split(self, X, y, num_samples, num_features):
        best_gini = float('inf')
        best_split = {}

        for feature_index in range(num_features):
            feature_values = X[:, feature_index]
            unique_values = np.unique(feature_values)

            for threshold in unique_values:
                left_indices = []
                right_indices = []
                for sample_index in range(num_samples):
                    if isinstance(threshold, str):  
                        condition = feature_values[sample_index] == threshold
                    else:
                        condition = feature_values[sample_index] <= threshold

                    if condition:
                        left_indices.append(sample_index)
                    else:
                        right_indices.append(sample_index)

                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue

                gini = self._calculate_gini_index(y, left_indices, right_indices)
                if gini < best_gini:
                    best_gini = gini
                    best_split = {'feature_index': feature_index,
                                  'threshold': threshold,
                                  'left_indices': left_indices,
                                  'right_indices': right_indices}

        return best_split

    def _calculate_gini_index(self, y, left_indices, right_indices):
        num_left = len(left_indices)
        num_right = len(right_indices)
        total = num_left + num_right

        gini_left = 1 - sum([(np.count_nonzero(y[left_indices] == c) / num_left) ** 2 for c in np.unique(y[left_indices])])
        gini_right = 1 - sum([(np.count_nonzero(y[right_indices] == c) / num_right) ** 2 for c in np.unique(y[right_indices])])

        gini = (num_left / total) * gini_left + (num_right / total) * gini_right
        return gini

    def predict(self, X):
        predictions = []
        for sample in X:
            predictions.append(self._predict_sample(sample, self.tree))
        return predictions

    def _predict_sample(self, sample, tree):
        if 'class' in tree:
            return tree['class']

        feature_value = sample[tree['feature_index']]
        if isinstance(feature_value, str):  
            condition = feature_value == tree['threshold']
        else:
            condition = feature_value <= tree['threshold']

        if condition:
            return self._predict_sample(sample, tree['left'])
        else:
            return self._predict_sample(sample, tree['right'])

    def print_tree(self, node, feature_names=None, class_names=None, indent=''):
      if 'class' in node:
        print(indent + f'Predict: {class_names[node["class"]]}')
      else:
        if 'is' in str(node["threshold"]):
            print(indent + f'{feature_names[node["feature_index"]]}')
        else:
            print(indent + f'├── {feature_names[node["feature_index"]]}')
        for value, child in [('is ' + str(node["threshold"]), node["left"]),
                             ('is not ' + str(node["threshold"]), node["right"])]:
            if 'Outlook' in str(node["feature_index"]) and 'is Sunny' in str(value):
                print(indent + f'├── {value}:')
            else:
                print(indent + f'└── {value}:')
            self.print_tree(child, feature_names, class_names, indent + '│   ' if 'is' in str(value) else indent + '    ')

X_train = np.array([
    ['Sunny', 'Hot', 'High', 'Weak'],
    ['Sunny', 'Hot', 'High', 'Strong'],
    ['Overcast', 'Hot', 'High', 'Weak'],
    ['Rain', 'Mild', 'High', 'Weak'],
    ['Rain', 'Cool', 'Normal', 'Weak'],
    ['Rain', 'Cool', 'Normal', 'Strong'],
    ['Overcast', 'Cool', 'Normal', 'Strong'],
    ['Sunny', 'Mild', 'High', 'Weak'],
    ['Sunny', 'Cool', 'Normal', 'Weak'],
    ['Rain', 'Mild', 'Normal', 'Weak'],
    ['Sunny', 'Mild', 'Normal', 'Strong'],
    ['Overcast', 'Mild', 'High', 'Strong'],
    ['Overcast', 'Hot', 'Normal', 'Weak'],
    ['Rain', 'Mild', 'High', 'Strong']
])

y_train = np.array(['No', 'No', 'Yes', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'Yes', 'Yes', 'Yes', 'Yes', 'No'])

clf = CARTDecisionTree(max_depth=5)
clf.fit(X_train, y_train)
print("Output:")
print("\n")
clf.print_tree(clf.tree, feature_names=clf.feature_names, class_names=clf.class_names)


Output:


├── Outlook
└── is Overcast:
│   Predict: Yes
└── is not Overcast:
│   ├── Humidity
│   └── is High:
│   │   ├── Outlook
│   │   └── is Rain:
│   │   │   ├── Wind
│   │   │   └── is Strong:
│   │   │   │   Predict: No
│   │   │   └── is not Strong:
│   │   │   │   Predict: Yes
│   │   └── is not Rain:
│   │   │   Predict: No
│   └── is not High:
│   │   ├── Wind
│   │   └── is Strong:
│   │   │   ├── Outlook
│   │   │   └── is Rain:
│   │   │   │   Predict: No
│   │   │   └── is not Rain:
│   │   │   │   Predict: Yes
│   │   └── is not Strong:
│   │   │   Predict: Yes
